In [134]:
from IPython.display import HTML
HTML("<br/><b style='font-size:30px;margin:5%;;'>sudo apt-get install ipython3</b><br/><br/><b style='font-size:30px;margin:5%;;'>ipython3 notebook</b>")

In [145]:
# log - fonksiyonun adı, çalışma tarihini ve çalışma süresini yaz
from functools import wraps, partial
import datetime, time
def log(func=None, *, prefix=""):
    if func is None: return partial(log, prefix=prefix)
    # func -> Loglanacak fonksiyon
    @wraps(func) # functaki özellikleri wrapper'a aktarır
    def wrapper(*args, **kwargs):
        start = time.clock()
        try:
            return func(*args, **kwargs)
        finally:
            stop = time.clock()
            print("{} {} fonksiyonu {} tarihinde çalıştırıldı ve {}sn sürdü".format(
                    prefix,
                    func.__qualname__,
                    datetime.date.today(),
                    stop-start
                ))
    return wrapper

def clslog(cls, prefix=""):
    for key, val in vars(cls).items():
        if callable(val):
            setattr(cls, key, log(val, prefix=prefix))
    return cls

# type('A', (), {})

class generic_type(type):
    
    def __new__(cls, name, parents, clsdict):
        print("cls logger")
        clsobj = super().__new__(cls, name, parents, clsdict)
        new = clslog(clsobj)
        return new

In [146]:
class User:

    def __init__(self, name, email, age, passwd):
        self.name = name
        self.email = email
        self.age = age
        self.passwd = passwd

class Point:

    def __init__(self, x, y):
        self.x = x
        self.y = y

class Connection:
    
    def __init__(self, user, hostname, port):
        self.user = user
        self.hostname = hostname
        self.port = port
    

In [147]:
u = User(name='mehmet', email='@gmail.com', age=23, passwd='asd')

In [148]:
u.__dict__

{'age': 23, 'email': '@gmail.com', 'name': 'mehmet', 'passwd': 'asd'}

In [149]:
from inspect import Signature, Parameter

def make_signature(fields):
    return Signature(Parameter(name, Parameter.POSITIONAL_OR_KEYWORD) for name in fields)

In [140]:
from collections import OrderedDict

class struct_meta(type):
    @classmethod
    def __prepare__(cls, name, bases):
        return OrderedDict()
    
    def __new__(cls, clsname, bases, clsdict):
        fields = [key for key, val in clsdict.items() 
                  if isinstance(val, Descriptor)]
        for name in fields:
            clsdict[name].name = name
            
        clsobj = super().__new__(cls, clsname, bases, dict(clsdict))
        clsobj = clslog(clsobj)
        sign = make_signature(fields)
        setattr(clsobj, "__signature__", sign)
        return clsobj

In [141]:
class Struct(metaclass=struct_meta):
    _fields = []
    def __init__(self, *args, **kwargs):
        bound = self.__signature__.bind(*args, *kwargs)
        for key, value in bound.arguments.items():
            setattr(self, key, value)
            


In [142]:
class Descriptor:
    def __init__(self, name=None):
        self.name = name
    
    def __set__(self, instance, value):
        print("set", value)
        instance.__dict__[self.name]=value
        
    def __delete__(self, instance):
        print("del", self.name)
        del instance.__dict__[self.name]
        
class Typed(Descriptor):
    ty = object
    def __set__(self, instance, value):
        if not isinstance(value, self.ty):
            raise TypeError("Expected {}", self.ty)
        return super().__set__(instance, value)

class Integer(Typed):
    ty = int

class String(Typed):
    ty = str

class Float(Typed):
    ty = float

In [143]:
class User(Struct):
    name = String()
    email = String()
    age = Integer()
    passwd = Integer()
    
class Point(Struct):
    x = Float()
    y = Float()
    
class Connection(Struct):
    user = String()
    hostname = String()
    port = Integer()

In [144]:
u = User('mehmet', '@hotmail.com', 12, 23)

set mehmet
set @hotmail.com
set 12
set 23
 Struct.__init__ fonksiyonu 2016-10-21 tarihinde çalıştırıldı ve 9.999999999976694e-05sn sürdü
